# 大模型编码demo学习

In [ ]:
print("Hello, World!")

## 一、 使用阿里云百炼调用大模型 key base-url model
注意：
1. 在jupyter中似乎无法读环境变量，因此key使用明文，正常应该使用环境变量
    

In [ ]:
import os
from openai import OpenAI


# 打印环境变量
print(os.getenv("DASHSCOPE_API_KEY"))


client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    #打印环境变量
    api_key="sk-730784af62d14bc6b5067a2e6633f096",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "你是谁？"},
    ],
    # Qwen3模型通过enable_thinking参数控制思考过程（开源版默认True，商业版默认False）
    # 使用Qwen3开源版模型时，若未启用流式输出，请将下行取消注释，否则会报错
    # extra_body={"enable_thinking": False},
)
print(completion.model_dump_json())

## 二、 使用langchain调用大模型
注意：
1. 因为langchain升级很快，会有很多东西过时，学习教程中的导入包以及predict方法无法使用，需要时刻注意官网的更新

In [ ]:
# langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

api_key="sk-730784af62d14bc6b5067a2e6633f096"
base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"

llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0,
    api_key=api_key, 
    base_url=base_url)

prompt = PromptTemplate.from_template("你是一个起名字大师，请模仿实例起三个{country}名字，比如男孩名字经常叫做{boy_name}，女孩名字经常叫做{girl_name}")
message = prompt.format(country="中国特色的", boy_name="高义", girl_name="白洁")

print(message)
response = llm.invoke(message)
print(response.content)

## 三、标准输出格式
1. 可自定义输出解析器 继承BaseOutputParser
2. temperature最好为0，防止模型输出格式的变化
3. 用于对数据的格式清洗 todo 代码

In [ ]:
# todo 标准输出格式
# 1. 可自定义输出解析器 继承BaseOutputParser
# 2. temperature最好为0，防止模型输出格式的变化
# 3. 用于对数据的格式清洗 todo 代码


## 四、模型IO 大语言模型的交互接口
- prompts ： 模版化
- Language models ：LLM 和 chat model 
- Ourput parser ： 格式处理，比如处理为JSON和其他系统对接

### prompts 模版 提示词提炼为模版，实现提示词的复用、版本管理、动态变化

优秀提示词：
- 角色：引导AI进入具体场景，赋予其身份
- 问题：描述问题和困惑，以及背景信息
- 目标：描述需求，希望达成的目标
- 要求：告诉AI回答是注意什么，或者如何回复

比如 ：我想去旅游，有什么建议？可以拓展为：
我想旅游，请根据以下条件给我推荐几个目的地和行程安排：
- 出发地：[你的城市]
- 预计时间：[几天]，大致出发时间：[月份或节假日]
- 预算：[金额范围]
- 偏好：[自然风光 / 人文历史 / 海边度假 / 城市漫步 / 少人安静 等]
- 出行方式：[飞机 / 高铁 / 自驾 / 无限制]
- 同行人员：[一个人 / 情侣 / 家庭带小孩 / 老人 等]

请给出推荐的目的地、适合的时间、交通方式、住宿建议和注意事项。


### PromptTemplate 字符串模版，对应LLM

In [ ]:
from langchain.prompts import PromptTemplate
import os

prompt = PromptTemplate(
    input_variables=["country"],
    template="What is the capital of {country}?"
)

prompt.format_prompt(country="China")

### ChatPromptTemplate 对话模版具有结构，对应chat model
- 角色 system user ai
- 对话模式

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant,your name is {name}"),
    ("user", "hello,{name}"),
    ("ai", "hi"),
    ("user", "What is the capital of {country}?"),
    ("ai", "The capital of {country} is {capital}.")
])

prompt.format_messages(country="China", name="John", capital="Beijing")

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

system_message = SystemMessage(content="You are a helpful assistant,your name is {name}",additional_kwargs={"name":"John"})
human_message = HumanMessage(content="hello,{name}",additional_kwargs={"name":"John"})
ai_message = AIMessage(content="hi",additional_kwargs={"name":"John"})
human_message = HumanMessage(content="What is the capital of {country}?",additional_kwargs={"country":"China"})
ai_message = AIMessage(content="The capital of {country} is {capital}.",additional_kwargs={"country":"China","capital":"Beijing"})

messages = [system_message,human_message,ai_message,human_message,ai_message]

print(messages)




In [ ]:
# ChatMessagePromptTemplate  role 自定义角色
from unittest import result
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "你真是个{subject}，{subject}真是个{subject}"

result = ChatMessagePromptTemplate.from_template(role="user",template=prompt) 

result.format(subject="猪猪")

In [ ]:
# AIMessagePromptTemplate 
from unittest import result
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "你真是个{subject}，{subject}真是个{subject}"

result = AIMessagePromptTemplate.from_template(template=prompt) 

result.format(subject="猪猪")

### 自定义模版


In [ ]:
## 函数大师 ： 根据函数名称寻找函数，并给出中文说明

from langchain.prompts import StringPromptTemplate

def hello_world():
    print("hello world")
    return "hello world"


PROMPT = """\
你是一个非常有经验的程序员，现在给你如下函数，你会严格按照如下格式，输出这段代码的函数名称、源代码、中文解释。
函数名称：{function_name}
源代码：{source_code}
中文解释：
"""

import inspect

def get_source_code(function_name):
    return inspect.getsource(function_name)

# 自定义模版class


class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 获取函数源代码
        source_code  = get_source_code(kwargs["function_name"])
        # 生成提示词模版
        prompt = PROMPT.format(function_name=kwargs["function_name"].__name__,source_code=source_code)
        return prompt
    
a = CustomPromptTemplate(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

# 和LLM交互
from langchain_openai import ChatOpenAI
import os

api_key="sk-730784af62d14bc6b5067a2e6633f096"
base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"

llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0,
    api_key=api_key, 
    base_url=base_url)

response = llm.invoke(pm)

print(response.content)





### jinja2 与f-string

In [ ]:
# f-string 格式化字符串 python内置的模版引擎
from langchain.prompts import PromptTemplate

f_string = """
请给我讲一个关于{topic}的故事，故事要包含{topic}的{detail}
"""

prompt = PromptTemplate.from_template(f_string)

print(prompt.format(topic="张三",detail="身世"))

In [ ]:
# jinjia2 模版引擎
! pip install jinja2

In [ ]:
# jinjia2 模版引擎 更为灵活生成各种标记格式的文档
from langchain.prompts import PromptTemplate

jinjia2_stirng = "请给我讲一个关于{{topic}}的故事，故事要包含{{topic}}的{{detail}}"

prompt = PromptTemplate.from_template(jinjia2_stirng,template_format="jinja2")

print(prompt.format(topic="张三",detail="身世"))

### 三层提示词设计 性格-行为-禁止
- 组合模版 
- 多层模版 
- Final prompt & Pipeline prompts


In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
PROMPT = """
你是一个经验丰富的程序员，你是中国人，住在北京。
你总是穿格子衫。
你从不说脏话。
"""



In [ ]:

from langchain.prompts import PromptTemplate

# 子模板定义
character_prompt = PromptTemplate.from_template("你是一个经验丰富的{person}，你是{country}人，住在{city}。")
behavior_prompt = PromptTemplate.from_template("你总是穿{clothes}。")
forbidden_prompt = PromptTemplate.from_template("你从不{forbidden}。")

# 最终模板
final_prompt = PromptTemplate.from_template("""
{character}
{behavior}
{forbidden}
""")

# 模板列表
pipeline_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("forbidden", forbidden_prompt)
]

# 输入参数
input_values = {
    "person": "导游",
    "country": "中国",
    "city": "成都",
    "clothes": "唐装",
    "forbidden": "说脏话"
}

# 用于保存每一层输出
intermediate_values = {}

# 使用循环逐步构建中间变量
for name, prompt in pipeline_prompts:
    intermediate_values[name] = prompt.format(**input_values)

print(intermediate_values)

# 最终生成完整提示
final_result = final_prompt.format(**intermediate_values)

# 输出
print(final_result)




